In [7]:
import pandas as pd
%matplotlib inline
import os
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
import gc
from tqdm import tqdm
import pickle
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import hstack, vstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# from skopt.space import Integer, Categorical, Real, Log10
# from skopt.utils import use_named_args
# from skopt import gp_minimize
from gensim.models import Word2Vec, FastText
import gensim
import re

In [2]:
# 读取 设备信息，包括品牌和型号
deviced_brand=pd.read_csv('./Demo/deviceid_brand.tsv',sep='\t', names=['device_id','brand','model'])
# 读取 app 信息，包括 app 所属的类别
package_label=pd.read_csv('./Demo/package_label.tsv',sep='\t',names=['app','class1','class2'])
# 读取训练数据集
deviceid_train=pd.read_csv('./Demo/deviceid_train.tsv',sep='\t',names=['device_id','sex','age'])
# 读取测试数据集
deviceid_test=pd.read_csv('./Demo/deviceid_test.tsv',sep='\t',names=['device_id'])
# 读取 app 数据
app_category=pd.read_csv('./Demo/package_label.tsv',sep='\t', names=['app', 'category', 'app_name'])
# 读取设备安装的 app 数据
deviceid_packages=pd.read_csv('./Demo/deviceid_packages.tsv',sep='\t', names=['device_id','apps'])
# 读取 APP 使用情况
package_time=pd.read_csv('./Demo/deviceid_package_start_close.tsv',sep='\t',names=['device_id','app','start','close'])

## 用户 APP 使用行为特征构造

In [3]:
# 计算 app 使用时长，单位为秒
package_time['period']=(package_time['close']-package_time['start'])/1000
# 把 app 的开始使用时间戳转换为 pd.datetime 数据类型
package_time['start']=pd.to_datetime(package_time['start'], unit='ms')
# 删除 app 使用结束的时间戳
del package_time['close']
gc.collect()

94

In [4]:
# 分别提取 app 开始使用的小时，日期，和星期几
package_time['hour']=package_time['start'].dt.hour
package_time['date']=package_time['start'].dt.date
package_time['dayofweek']=package_time['start'].dt.dayofweek

In [5]:
package_time.head()

,device_id,app,start,period,hour,date,dayofweek
0,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:05:23.086,10.014,2,2017-03-09,3
1,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:09:53.513,5.002,2,2017-03-09,3
2,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:11:03.551,1119.971,2,2017-03-09,3
3,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 04:41:04.940,19.984,4,2017-03-09,3
4,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 09:02:56.275,25.001,9,2017-03-09,3


### 统计每个设备分别在每天、每小时、每周几、每个 APP 上使用的时间

In [6]:
#计算每个设备的每天使用时间
dtime=package_time.groupby(['device_id','date'])['period'].agg('sum')
#计算每个设备的每小时使用时间
qtime=package_time.groupby(['device_id','hour'])['period'].agg('sum')
#计算每个设备的每周几使用时间
wtime=package_time.groupby(['device_id','dayofweek'])['period'].agg('sum')
#计算每个设备上每个 app 的使用时间
atime=package_time.groupby(['device_id','app'])['period'].agg('sum')

In [7]:
dtime.head()

device_id                         date      
00009270c4ec26e1d76f5d86847009c9  2017-02-28    37312.432
                                  2017-03-01       22.518
                                  2017-03-02     5264.417
                                  2017-03-03      642.991
                                  2017-03-04       12.237
Name: period, dtype: float64

### 统计每个设备使用 APP 数量的方差、平均值、最大值

In [8]:
# 计算每个设备每天使用的 app 数量
# 首先根据 ['device_id', 'date'] 分组，然后将每个设备每天的所有 app 用空格连接起来
dapp=package_time[['device_id', 'date', 'app']].drop_duplicates().groupby(['device_id', 'date'])['app'].agg(' '.join)
dapp = dapp.reset_index()
# 根据空格分隔 app，并计算数量
dapp['app_nums']=dapp['app'].apply(lambda x: x.split(' ')).apply(len)

In [9]:
dapp.head()

,device_id,date,app,app_nums
0,00009270c4ec26e1d76f5d86847009c9,2017-02-28,1896072db9ce6406febfc17f681c2086,1
1,00009270c4ec26e1d76f5d86847009c9,2017-03-01,1896072db9ce6406febfc17f681c2086,1
2,00009270c4ec26e1d76f5d86847009c9,2017-03-02,1896072db9ce6406febfc17f681c2086,1
3,00009270c4ec26e1d76f5d86847009c9,2017-03-03,1896072db9ce6406febfc17f681c2086,1
4,00009270c4ec26e1d76f5d86847009c9,2017-03-04,1896072db9ce6406febfc17f681c2086,1


In [10]:
# 计算每个设备使用 APP 数量的方差、平均值、最大值。 agg() 中可以传入多个函数对 groupby 对象的操作函数
dapp_stat = dapp.groupby('device_id')['app_nums'].agg(
    {'std': 'std', 'mean': 'mean', 'max': 'max'})
dapp_stat = dapp_stat.reset_index()
dapp_stat.columns = ['device_id', 'app_num_std', 'app_num_mean', 'app_num_max']

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
dapp_stat.head()

,device_id,app_num_std,app_num_mean,app_num_max
0,00009270c4ec26e1d76f5d86847009c9,0.461133,1.166667,3
1,000189ef5d5b951841d416a8c6c5b995,1.565763,9.500000,12
2,00026d79a6f0955fc860947724e24765,0.000000,1.000000,1
3,0002e3afb8146bc08e40575e45f0eca6,1.000000,2.000000,3
4,0004709a296f9b925ae283efe2f043e7,2.366120,5.206897,11


### 统计每个设备使用时间的和、方差、平均值、最大值

In [12]:
dtime = dtime.reset_index()
# 和上面的操作类似，计算每个设备使用时间的和、方差、平均值、最大值
dtime_stat = dtime.groupby(['device_id'])['period'].agg(
    {'sum': 'sum', 'mean': 'mean', 'std': 'std', 'max': 'max'}).reset_index()
dtime_stat.columns = ['device_id', 'date_sum',
                      'date_mean', 'date_std', 'date_max']

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.


In [13]:
dtime_stat.head()

,device_id,date_sum,date_mean,date_std,date_max
0,00009270c4ec26e1d76f5d86847009c9,334102.661,11136.755367,18704.949809,63619.553
1,000189ef5d5b951841d416a8c6c5b995,1099796.807,34368.650219,25602.049326,93056.309
2,00026d79a6f0955fc860947724e24765,669.056,223.018667,318.287555,588.807
3,0002e3afb8146bc08e40575e45f0eca6,476878.857,68125.551000,161844.346716,433661.470
4,0004709a296f9b925ae283efe2f043e7,413026.398,14242.289586,11618.480923,49783.284


In [14]:
qtime.reset_index().head(1)

,device_id,hour,period
0,00009270c4ec26e1d76f5d86847009c9,0,32.902


In [15]:
# 通过透视把每个设备的每个小时的使用时间的转置。变换之前的数据每个设备有 24 行，每行表示设备每个小时的使用时间。
# 变换之后的数据有 24 列，每一列表示设备每个小时的使用时间
qtime = qtime.reset_index()
ftime = qtime.pivot(index='device_id', columns='hour',
                    values='period').fillna(0)
# 设置列名
ftime.columns = ['h%s' % i for i in range(24)]
ftime.reset_index(inplace=True)

In [16]:
ftime.head()

,device_id,h0,h1,h2,h3,h4,h5,h6,h7,h8,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,00009270c4ec26e1d76f5d86847009c9,32.902,3120.354,19409.942,80.586,44333.680,6281.700,21345.857,72144.487,752.370,...,60830.868,23066.936,28958.591,0.000,0.000,0.000,0.000,760.276,916.438,2579.052
1,000189ef5d5b951841d416a8c6c5b995,38946.746,60793.584,49900.472,29420.010,78586.490,4256.499,9826.159,47229.344,66209.618,...,122468.084,59922.699,151795.372,30074.698,319.986,9457.775,5928.054,11045.564,5461.401,9960.791
2,00026d79a6f0955fc860947724e24765,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,71.046
3,0002e3afb8146bc08e40575e45f0eca6,1252.860,0.000,1207.510,656.173,277.510,20.466,889.112,431349.747,79.942,...,34971.811,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0004709a296f9b925ae283efe2f043e7,19263.344,43943.525,35355.233,19295.768,21957.694,14968.913,23465.262,17109.129,18054.109,...,8517.143,5339.508,6430.394,585.658,0.000,7827.382,9094.190,13483.280,35541.093,27232.087


In [17]:
wtime = wtime.reset_index()
# 通过透视把每个设备的每个周几的使用时间的转置。变换之前的数据每个设备有 7 行，每行表示设备每个周几的使用时间。
# 变换之后的数据有 7 列，每一列表示设备每个周几的使用时间
weektime = wtime.pivot(
    index='device_id', columns='dayofweek', values='period').fillna(0)
weektime.columns = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
weektime.reset_index(inplace=True)

In [18]:
weektime.head()

,device_id,w0,w1,w2,w3,w4,w5,w6
0,00009270c4ec26e1d76f5d86847009c9,24704.498,138049.645,55275.594,11877.301,27579.032,65830.257,10786.334
1,000189ef5d5b951841d416a8c6c5b995,126464.294,231742.053,137188.344,169735.847,206564.846,161872.151,66229.272
2,00026d79a6f0955fc860947724e24765,9.203,71.046,0.000,0.000,0.000,0.000,588.807
3,0002e3afb8146bc08e40575e45f0eca6,0.000,435997.167,94.055,0.000,257.875,39759.475,770.285
4,0004709a296f9b925ae283efe2f043e7,22878.187,67882.311,38606.127,79848.598,85544.656,79216.949,39049.570


In [19]:
atime.reset_index().head(1)

,device_id,app,period
0,00009270c4ec26e1d76f5d86847009c9,1896072db9ce6406febfc17f681c2086,330562.65


###  找出每个设备上使用时间最多的 app 的行索引

In [20]:
# 找出每个设备上使用时间最多的 app 的行索引
atime = atime.reset_index()
app = atime.groupby(['device_id'])['period'].idxmax()
app.head()

device_id
00009270c4ec26e1d76f5d86847009c9     0
000189ef5d5b951841d416a8c6c5b995    10
00026d79a6f0955fc860947724e24765    22
0002e3afb8146bc08e40575e45f0eca6    25
0004709a296f9b925ae283efe2f043e7    36
Name: period, dtype: int64

In [21]:
# 把表连接起来，得到用户行为数据表
# dapp_stat 是每个设备每天使用 APP 数量的方差、平均值、最大值
# dtime_stat 是每个设备使用时间的和、方差、平均值、最大值
user = pd.merge(dapp_stat, dtime_stat, on='device_id', how='left')
# ftime 是每个设备的每个小时的使用时间
user = pd.merge(user, ftime, on='device_id', how='left')
# weektime 是每个设备的每个周几的使用时间
user = pd.merge(user, weektime, on='device_id', how='left')
# atime.iloc[app] 表示每个设备使用时间最多的app 的名字和时间
user = pd.merge(user, atime.iloc[app], on='device_id', how='left')

In [22]:
user.head()

,device_id,app_num_std,app_num_mean,app_num_max,date_sum,date_mean,date_std,date_max,h0,h1,...,h23,w0,w1,w2,w3,w4,w5,w6,app,period
0,00009270c4ec26e1d76f5d86847009c9,0.461133,1.166667,3,334102.661,11136.755367,18704.949809,63619.553,32.902,3120.354,...,2579.052,24704.498,138049.645,55275.594,11877.301,27579.032,65830.257,10786.334,1896072db9ce6406febfc17f681c2086,330562.650
1,000189ef5d5b951841d416a8c6c5b995,1.565763,9.500000,12,1099796.807,34368.650219,25602.049326,93056.309,38946.746,60793.584,...,9960.791,126464.294,231742.053,137188.344,169735.847,206564.846,161872.151,66229.272,5e8709466b22da6b45bfd30825bd3620,621111.263
2,00026d79a6f0955fc860947724e24765,0.000000,1.000000,1,669.056,223.018667,318.287555,588.807,0.000,0.000,...,71.046,9.203,71.046,0.000,0.000,0.000,0.000,588.807,c33b35d6254ad9c0c238233eb97a6c60,669.056
3,0002e3afb8146bc08e40575e45f0eca6,1.000000,2.000000,3,476878.857,68125.551000,161844.346716,433661.470,1252.860,0.000,...,0.000,0.000,435997.167,94.055,0.000,257.875,39759.475,770.285,97d0422a3317b1929926dc90cda4fc53,471532.292
4,0004709a296f9b925ae283efe2f043e7,2.366120,5.206897,11,413026.398,14242.289586,11618.480923,49783.284,19263.344,43943.525,...,27232.087,22878.187,67882.311,38606.127,79848.598,85544.656,79216.949,39049.570,86f9f299cdbc8e2a19fed1712f522c49,346571.338


### 预处理 APP 类别，加上编码

In [23]:
# 计算每种类别的数量，并且添加编号 idx，结果中类别名作为索引
cat_enc = pd.DataFrame(app_category['category'].value_counts())
cat_enc['idx']=range(cat_enc.shape[0])
cat_enc.head()

,category,idx
系统工具,1005,0
金融,976,1
教育,843,2
其它,776,3
休闲,625,4


In [24]:
# 使用 map 函数，将每个类别对应的编号映射到原来的 app 表中
app_category['cat_enc'] = app_category['category'].map(cat_enc['idx'])
app_category.head()

,app,category,app_name,cat_enc
0,8c4ac9e7f1d13362a7a443d9fe385d48,实用工具,日历应用,9
1,991a8d9248a0b224c8cdc6feda5c70a9,视频,网络电视,10
2,b255f4de409d3f905b8432e6564cd243,母婴亲子,孕育工具助手,39
3,215263e647821ceb627c55a27475871a,金融,投资理财,1
4,92b700e7f608dbfe4247809dc6e6dfd0,其它,其它,3


In [25]:
# 将 app 所在的列设置为索引
app_category.set_index(['app'], inplace=True)

In [26]:
# 将每个设备的每个 APP 的使用时间的表添加编号映射，如果没有编号，则填入 45
atime['app_cat_enc'] = atime['app'].map(app_category['cat_enc']).fillna(45)

In [27]:
atime.head()

,device_id,app,period,app_cat_enc
0,00009270c4ec26e1d76f5d86847009c9,1896072db9ce6406febfc17f681c2086,330562.650,6.0
1,00009270c4ec26e1d76f5d86847009c9,8c8544b6c129ad4a431be753143ed1c3,15.585,28.0
2,00009270c4ec26e1d76f5d86847009c9,90cb852cf345e04d508fe03f74089183,3524.426,45.0
3,000189ef5d5b951841d416a8c6c5b995,039be717f253f7b10ed1ce405de08b9f,22.455,45.0
4,000189ef5d5b951841d416a8c6c5b995,0b0816ff97e9a3e5501ed2dcb4a0d66e,69578.566,45.0


### 统计每个设备在每个APP类别分别使用了多少个 APP

In [28]:
# 计算每个设备在每个APP类别分别使用了多少个 APP
cat_num = atime.groupby(['device_id', 'app_cat_enc'])['app'].agg('count').reset_index()

In [29]:
# 计算每个设备在每个APP类别分别使用了多长时间
cat_time = atime.groupby(['device_id', 'app_cat_enc'])[
    'period'].agg('sum').reset_index()

In [30]:
cat_num.head()

,device_id,app_cat_enc,app
0,00009270c4ec26e1d76f5d86847009c9,6.0,1
1,00009270c4ec26e1d76f5d86847009c9,28.0,1
2,00009270c4ec26e1d76f5d86847009c9,45.0,1
3,000189ef5d5b951841d416a8c6c5b995,6.0,1
4,000189ef5d5b951841d416a8c6c5b995,7.0,2


In [31]:
# 通过透视来转置。变换之后的数据有 46 列，每一列表示某个列别的 APP 个数。
app_cat_num = cat_num.pivot(index='device_id', columns='app_cat_enc', values='app').fillna(0)
app_cat_num.columns = ['cat%s' % i for i in range(46)]
app_cat_num.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45
device_id,,,,,,,,,,,,,,,,,,,,,
00009270c4ec26e1d76f5d86847009c9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000189ef5d5b951841d416a8c6c5b995,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
00026d79a6f0955fc860947724e24765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0002e3afb8146bc08e40575e45f0eca6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
0004709a296f9b925ae283efe2f043e7,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0


### 统计每个设备在每个APP类别分别使用了多长时间

In [32]:
# 通过透视来转置每个设备在每个APP类别分别使用了多长时间。变换之后的数据有 46 列，每一列表示设备在某个列别的使用时长。
app_cat_time=cat_time.pivot(index='device_id', values='period', columns='app_cat_enc').fillna(0)
app_cat_time.columns = ['time%s' % i for i in range(46)]
app_cat_time.head()

,time0,time1,time2,time3,time4,time5,time6,time7,time8,time9,...,time36,time37,time38,time39,time40,time41,time42,time43,time44,time45
device_id,,,,,,,,,,,,,,,,,,,,,
00009270c4ec26e1d76f5d86847009c9,0.000,0.00,0.0,0.0,0.0,0.000,330562.650,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3524.426
000189ef5d5b951841d416a8c6c5b995,0.000,0.00,0.0,0.0,0.0,0.000,144465.215,622228.87,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,330802.368
00026d79a6f0955fc860947724e24765,0.000,0.00,0.0,0.0,0.0,0.000,0.000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,669.056
0002e3afb8146bc08e40575e45f0eca6,0.000,0.00,0.0,0.0,0.0,0.000,5213.155,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,471665.702
0004709a296f9b925ae283efe2f043e7,80.021,885.89,0.0,0.0,0.0,19890.917,18157.668,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370306.639


In [33]:
# 合并上面的两张表，并保存到 user_behavior.csv
user = pd.merge(user, app_cat_num, on='device_id', how='left')
user = pd.merge(user, app_cat_time, on='device_id', how='left')
user.to_csv('Demo/user_behavior.csv', index=False)

### 查看内存占用，优化内存使用

In [34]:
import pandas as pd
from sys import getsizeof
def get_memory(threshold=1048576):
    '''查看变量占用内存情况

    :param threshold: 仅显示内存数值大于等于threshold的变量, 默认为 1MB=1048576B
    '''
    memory_df=pd.DataFrame(columns=['name', 'memory', 'convert_memory'])
    i=0
    for key in list(globals().keys()):
        memory = eval("getsizeof({})".format(key))
        if memory<threshold:
            continue
        if(memory>1073741824):# GB
            unit='GB'
            convert_memory=round(memory/1073741824)
        elif(memory>1048576):# MB
            unit='MB'
            convert_memory=round(memory/1048576)           
        elif(memory>1024):# KB
            unit='KB'
            convert_memory=round(memory/1024)  
        else:
            unit='B' 
            convert_memory = memory
        memory_df.loc[i]=[key, memory, str(convert_memory)+unit]
        i=i+1
    # 按照内存占用大小降序排序    
    memory_df.sort_values("memory",inplace=True,ascending=False)
    return memory_df
    
memory_df = get_memory()
memory_df

,name,memory,convert_memory
6,package_time,9180235152,9GB
12,dapp,438686771,418MB
7,dtime,185835035,177MB
10,atime,148545564,142MB
8,qtime,125594117,120MB
18,user,89745142,86MB
9,wtime,45867197,44MB
5,deviceid_packages,35813634,34MB
22,app_cat_time,33236263,32MB
21,app_cat_num,33236263,32MB


In [35]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        # 如果当前列的类型不是 object，剩下就是 int,int32, float,float32 等
        if col_type != object:
            # 取出这一列的最小值
            c_min = df[col].min()
            # 取出这一列的最大值
            c_max = df[col].max()
            # 如果类型是 int 开头的
            if str(col_type)[:3] == 'int':
                # 如果最大值和最小值都在 int8 的范围内，则转为 int8
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 如果最大值和最小值都在 int16 的范围内，则转为 int16
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 如果最大值和最小值都在 int32 的范围内，则转为 int32
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 如果最大值和最小值都在 int64 的范围内，则转为 int64
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif str(col_type)[:3] == 'float':# 如果类型是 float 开头的
                # 如果最大值和最小值都在 float16 的范围内，则转为 float16
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                # 如果最大值和最小值都在 float32 的范围内，则转为 float32
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / (1024 ** 2) # 计算用了多少 M
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [36]:
package_time = reduce_mem(package_time)

1961.11 Mb, 1470.83 Mb (25.00 %)


In [37]:
# 删除变量，回收内存
del dapp
del dtime
del qtime
del dtime_stat
gc.collect()

140

## 手机属性特征构造

In [38]:
package_time.head()

,device_id,app,start,period,hour,date,dayofweek
0,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:05:23.086,10.014,2,2017-03-09,3
1,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:09:53.513,5.002,2,2017-03-09,3
2,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 02:11:03.551,1119.971,2,2017-03-09,3
3,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 04:41:04.940,19.984,4,2017-03-09,3
4,e0450666692b72a1f580dfa082e8b2ae,1896072db9ce6406febfc17f681c2086,2017-03-09 09:02:56.275,25.001,9,2017-03-09,3


###  统计每个设备在前 100 个 APP的使用时间

In [39]:
# 计算每个 APP 使用的时长总和，并取出使用时间最长的前 100 个 APP 的名称
app_use_time = package_time.groupby(['app'])['period'].agg('sum').reset_index()
app_use_top100 = app_use_time.sort_values(by='period', ascending=False)[:100]['app']

In [40]:
# 把 app 列设置为索引，取出使用时间最长的前 100 个
use_time_top100_statis=atime.set_index('app').loc[list(app_use_top100)].reset_index()

In [41]:
use_time_top100_statis.head()

,app,device_id,period,app_cat_enc
0,1896072db9ce6406febfc17f681c2086,00009270c4ec26e1d76f5d86847009c9,330562.650,6.0
1,1896072db9ce6406febfc17f681c2086,000189ef5d5b951841d416a8c6c5b995,144465.215,6.0
2,1896072db9ce6406febfc17f681c2086,0002e3afb8146bc08e40575e45f0eca6,5213.155,6.0
3,1896072db9ce6406febfc17f681c2086,0004709a296f9b925ae283efe2f043e7,18157.668,6.0
4,1896072db9ce6406febfc17f681c2086,000536b155abcf97971a6b99a6d7dbef,261031.006,6.0


In [42]:
# 通过透视来转置每个设备在前 100 个 APP的使用时间。变换之后的数据有 100 列，每一列表示设备在这个 APP 使用的时间。
top100_statis = use_time_top100_statis.pivot(
    index='device_id', columns='app', values='period').reset_index().fillna(0)
top100_statis.shape

(71843, 101)

In [43]:
deviced_brand.head()

,device_id,brand,model
0,f69cdc2f5018aa6d3d6500976b926c09,samsung,GT-N7100
1,522690089a93278b65025127d937e9bb,samsung,GT-I9507V
2,dcf13c9ec255153bfa909dc3c64908c3,Sony,L39u
3,58b5f0ec136407caf3708a4b05c1894f,Sony,L39u
4,dbca7cb790645b18383f4536e7f1f303,Sony,XM50h


### 统计每个设备所属的`品牌_型号`的数量

In [44]:
# 如果品牌名有空格，则取前面的作为新品牌名
deviced_brand.brand=deviced_brand.brand.astype(str).apply(lambda x:x.split(' ')[0].upper())

In [45]:
# 把品牌和型号拼接起来，作为新的字段
deviced_brand['ph_ver'] = deviced_brand['brand'] + '_' + deviced_brand['model']

In [46]:
# 统计品牌_型号的数量
ph_ver = deviced_brand['ph_ver'].value_counts()
ph_ver_cnt = pd.DataFrame(ph_ver).reset_index()
ph_ver_cnt.columns = ['ph_ver', 'ph_ver_cnt']
# 和原表拼接起来
deviced_brand = pd.merge(left=deviced_brand, right=ph_ver_cnt, on='ph_ver')

In [47]:
deviced_brand.head()

,device_id,brand,model,ph_ver,ph_ver_cnt
0,f69cdc2f5018aa6d3d6500976b926c09,SAMSUNG,GT-N7100,SAMSUNG_GT-N7100,1320
1,7f0119f89b0874947ad11de689744910,SAMSUNG,GT-N7100,SAMSUNG_GT-N7100,1320
2,afc87cbe98e92af973fbd06016e7dd75,SAMSUNG,GT-N7100,SAMSUNG_GT-N7100,1320
3,e2b1082e60c79a5adc2ec17ae5d2e320,SAMSUNG,GT-N7100,SAMSUNG_GT-N7100,1320
4,7ecdb8fda13fbfb8ecfedd0a842bc50b,SAMSUNG,GT-N7100,SAMSUNG_GT-N7100,1320


In [48]:
# 针对长尾分布做的一点处理
mask = (deviced_brand.ph_ver_cnt < 100)
deviced_brand.loc[mask, 'ph_ver'] = 'other'

In [49]:
# 将手机属性数据和训练集、测试集合并
deviceid_train = pd.merge(deviced_brand[['device_id', 'ph_ver']],
                 deviceid_train, on='device_id', how='right')
deviceid_test = pd.merge(deviced_brand[['device_id', 'ph_ver']],
                deviceid_test, on='device_id', how='right')

In [50]:
# 对 ph_ver 进行 label encoder
deviceid_train['ph_ver'] = deviceid_train['ph_ver'].astype(str)
deviceid_test['ph_ver'] = deviceid_test['ph_ver'].astype(str)
ph_ver_le = preprocessing.LabelEncoder()
deviceid_train['ph_ver'] = ph_ver_le.fit_transform(deviceid_train['ph_ver'])
deviceid_test['ph_ver'] = ph_ver_le.transform(deviceid_test['ph_ver'])

### 合并训练集、测试集，构造标签

In [51]:
# 构造标签，并对标签进行 label encoder
deviceid_train['label'] = deviceid_train['sex'].astype(str) + '-' + deviceid_train['age'].astype(str)
label_le = preprocessing.LabelEncoder()
deviceid_train['label'] = label_le.fit_transform(deviceid_train['label'])

# 把测试集的标签进行处理，合并训练集和测试集
deviceid_test['sex'] = -1
deviceid_test['age'] = -1
deviceid_test['label'] = -1
data = pd.concat([deviceid_train, deviceid_test], ignore_index=True)

In [52]:

# 将 ph_ver 进行独热编码
ph_ver_dummy = pd.get_dummies(data['ph_ver'])
ph_ver_dummy.columns = ['ph_ver_%s' %i
                        for i in range(ph_ver_dummy.shape[1])]
data = pd.concat([data, ph_ver_dummy], axis=1)
del data['ph_ver']
deviceid_train = data[data['sex']!=-1]
deviceid_test = data[data['sex']==-1]
del data
gc.collect()

421

### 统计每个app的总使用次数

In [53]:
# 统计每个app的总使用次数
app_num = package_time['app'].value_counts().reset_index()
app_num.columns = ['app', 'app_num']
app_num.head()

,app,app_num
0,1896072db9ce6406febfc17f681c2086,12050342
1,07e967d75aab2f6a52c558695a572a7c,3557203
2,8d2448133beb3422f0f638bacf8f7051,1287254
3,e29eb7083bdf54af48352ffa979fc830,1153291
4,4b58ecb20fe0d5e7a823b7d95911166d,817326


In [54]:
# 将 APP 的使用次数和原表合并
package_time = pd.merge(left=package_time, right=app_num, how='inner',on='app')
# 同样的，针对长尾分布做些处理（尝试过不做处理，或换其他阈值，这个100的阈值的准确率最高）
package_time.loc[package_time.app_num < 100, 'app'] = 'other'

### 统计每台设备的app数量

In [55]:
# 统计每台设备的app数量
df_app = package_time[['device_id', 'app']]
apps = df_app.drop_duplicates().groupby(['device_id'])[
    'app'].apply(' '.join).reset_index()
apps['app_length'] = apps['app'].apply(lambda x: len(x.split(' ')))

# 这是另一种统计每台设备的app数量的方法
# df_app = package_time[['device_id', 'app']]
# apps = df_app.drop_duplicates().groupby(['device_id'])[
#     'app'].agg('count').reset_index()

In [56]:
apps.head()

,device_id,app,app_length
0,00009270c4ec26e1d76f5d86847009c9,1896072db9ce6406febfc17f681c2086 90cb852cf345e...,3
1,000189ef5d5b951841d416a8c6c5b995,1896072db9ce6406febfc17f681c2086 37de3dfb9d3bc...,19
2,00026d79a6f0955fc860947724e24765,c33b35d6254ad9c0c238233eb97a6c60,1
3,0002e3afb8146bc08e40575e45f0eca6,1896072db9ce6406febfc17f681c2086 07e967d75aab2...,3
4,0004709a296f9b925ae283efe2f043e7,1896072db9ce6406febfc17f681c2086 07e967d75aab2...,14


In [57]:
# 将每台设备的 app 数量和分别和训练集、测试集合并
deviceid_train = pd.merge(deviceid_train, apps, on='device_id', how='left')
deviceid_test = pd.merge(deviceid_test, apps, on='device_id', how='left')

### 统计每个设备在每周几使用时间的的最大值、最小值、和、方差

In [58]:
# 统计每个设备在每周几使用时间的的最大值、最小值、和、方差
weektime['week_max'] = weektime.max(axis=1)
weektime['week_min'] = weektime.min(axis=1)
weektime['week_sum'] = weektime.sum(axis=1)
weektime['week_std'] = weektime.std(axis=1)
weektime.head()

,device_id,w0,w1,w2,w3,w4,w5,w6,week_max,week_min,week_sum,week_std
0,00009270c4ec26e1d76f5d86847009c9,24704.498,138049.645,55275.594,11877.301,27579.032,65830.257,10786.334,138049.645,10786.334,482938.640,144217.018764
1,000189ef5d5b951841d416a8c6c5b995,126464.294,231742.053,137188.344,169735.847,206564.846,161872.151,66229.272,231742.053,66229.272,1397768.132,397342.643587
2,00026d79a6f0955fc860947724e24765,9.203,71.046,0.000,0.000,0.000,0.000,588.807,588.807,0.000,1257.863,427.895521
3,0002e3afb8146bc08e40575e45f0eca6,0.000,435997.167,94.055,0.000,257.875,39759.475,770.285,435997.167,0.000,912876.024,313009.429201
4,0004709a296f9b925ae283efe2f043e7,22878.187,67882.311,38606.127,79848.598,85544.656,79216.949,39049.570,85544.656,22878.187,521449.241,148736.388547


In [59]:
# 将用户行为数据和训练集、测试集合并
del user['app']
deviceid_train = pd.merge(deviceid_train, user, on='device_id', how='left')
deviceid_test = pd.merge(deviceid_test, user, on='device_id', how='left')

In [60]:
# 将每个设备在前 100 个 APP的使用时间的表和训练集、测试集合并
top100_statis.columns = ['device_id'] + ['top100_statis_' + str(i) for i in range(0, 100)]
deviceid_train = pd.merge(deviceid_train, top100_statis, on='device_id', how='left')
deviceid_test = pd.merge(deviceid_test, top100_statis, on='device_id', how='left')
# 把处理后的训练集、测试集保存到 csv 中
deviceid_train.to_csv("./Demo/train_statistic_feat.csv", index=False)
deviceid_test.to_csv("./Demo/test_statistic_feat.csv", index=False)

In [61]:
# 每个设备安装的 APP，使用空格分割
def get_str(df):
    res=""
    for i in df.split(','):
        res+=i+" "
    return res
deviceid_packages["str_app"]=deviceid_packages['apps'].apply(lambda x:get_str(x))

In [62]:
# 计算每个 APP 的 tf-idf
tfidf = TfidfVectorizer()
train_str_app=pd.merge(deviceid_train[['device_id']],deviceid_packages[["device_id",'str_app']],on="device_id",how="left")
test_str_app=pd.merge(deviceid_test[['device_id']],deviceid_packages[["device_id",'str_app']],on="device_id",how="left")
cntTf=tfidf.fit_transform(deviceid_packages['str_app'])
# 将训练集和测试集的 app 列表转换为 tf-idf。使用 .tocsr() 转换为稀疏矩阵，节省内存。
train_app = tfidf.transform(list(train_str_app['str_app'])).tocsr()
test_app = tfidf.transform(list(test_str_app['str_app'])).tocsr()

In [63]:
# 取出训练集和测试集的设备 id
all_id=pd.concat([deviceid_train[["device_id"]],deviceid_test[['device_id']]])
all_id.index=range(len(all_id))


### 使用 app 的tfidf 来预测性别，分别使用 7 种方法

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.metrics import mean_squared_error

In [65]:
# 使用 app 的tfidf 来预测性别，分别使用 7 种方法
df_stack = pd.DataFrame()
n_folds = 5
# 预测性别，由于原表是从 1 开始，因此-1，变为从 0 开始
sex = deviceid_train['sex']-1
print('lr stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))
sex_va = 0
kv=StratifiedKFold(n_splits=n_folds, random_state=1017)
# LogisticRegression
for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    clf = LogisticRegression(random_state=1017, C=8)
    clf.fit(train_app[tr], sex[tr])
    # 由于标签为 0 和 1，因此取第一列作为标签
    sex_va = clf.predict_proba(train_app[va])[:,1]
    sex_te=clf.predict_proba(test_app)[:,1]
    print('得分' + str(mean_squared_error(sex[va], clf.predict(train_app[va]))))
    stack_train[va,0]=sex_va
    stack_test[:,0]+=sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['pack_tfidf_lr_classfiy_{}'.format(sex)] = stack[:, 0]

print('SGDClassifier stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))
# SGDClassifier
for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    sgd = SGDClassifier(random_state=1017, loss='log')
    sgd.fit(train_app[tr], sex[tr])
    sex_va = sgd.predict_proba(train_app[va])[:,1]
    sex_te = sgd.predict_proba(test_app)[:,1]
    print('得分' + str(mean_squared_error(sex[va], sgd.predict(train_app[va]))))
    stack_train[va,0] = sex_va
    stack_test[:,0]+= sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_sgd_classfiy_{}'.format(sex)] = stack[:, 0]


print('PassiveAggressiveClassifier stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))
# PassiveAggressiveClassifier
for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    pac = PassiveAggressiveClassifier(random_state=1017)
    pac.fit(train_app[tr], sex[tr])
    sex_va = pac._predict_proba_lr(train_app[va])[:,1]
    sex_te = pac._predict_proba_lr(test_app)[:,1]
    print(sex_va)
    print('得分' + str(mean_squared_error(sex[va], pac.predict(train_app[va]))))
    stack_train[va,0] += sex_va
    stack_test[:,0] += sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_pac_classfiy_{}'.format(sex)] = stack[:, 0]


# RidgeClassifier
print('RidgeClassfiy stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))
for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    ridge = RidgeClassifier(random_state=1017)
    ridge.fit(train_app[tr], sex[tr])
    sex_va = ridge._predict_proba_lr(train_app[va])[:,1]
    sex_te = ridge._predict_proba_lr(test_app)[:,1]
    print(sex_va)
    print('得分' + str(mean_squared_error(sex[va], ridge.predict(train_app[va]))))
    stack_train[va,0] += sex_va
    stack_test[:,0] += sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_ridge_classfiy_{}'.format(sex)] = stack[:, 0]
    
    
# BernoulliNB
print('BernoulliNB stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))

for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    bnb = BernoulliNB()
    bnb.fit(train_app[tr], sex[tr])
    sex_va = bnb.predict_proba(train_app[va])[:,1]
    sex_te = bnb.predict_proba(test_app)[:,1]
    print(sex_va)
    print('得分' + str(mean_squared_error(sex[va], bnb.predict(train_app[va]))))
    stack_train[va,0] += sex_va
    stack_test[:,0] += sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_bnb_classfiy_{}'.format(sex)] = stack[:, 0]    
    
    
# MultinomialNB
print('MultinomialNB stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))

for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    mnb = MultinomialNB()
    mnb.fit(train_app[tr], sex[tr])
    
    sex_va = mnb.predict_proba(train_app[va])[:,1]
    sex_te = mnb.predict_proba(test_app)[:,1]
    print(sex_va)
    print('得分' + str(mean_squared_error(sex[va], mnb.predict(train_app[va]))))
    stack_train[va,0] += sex_va
    stack_test[:,0] += sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_mnb_classfiy_{}'.format(sex)] = stack[:, 0]    
    
  
# LinearSVC
print('LinearSVC stacking')
stack_train = np.zeros((len(deviceid_train), 1))
stack_test = np.zeros((len(deviceid_test), 1))
for i, (tr,va) in enumerate(kv.split(train_app,sex)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    lsvc = LinearSVC(random_state=1017)
    lsvc.fit(train_app[tr], sex[tr])
    sex_va = lsvc._predict_proba_lr(train_app[va])[:,1]
    sex_te = lsvc._predict_proba_lr(test_app)[:,1]
    print(sex_va)
    print('得分' + str(mean_squared_error(sex[va], lsvc.predict(train_app[va]))))
    stack_train[va,0] += sex_va
    stack_test[:,0] += sex_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack['tfidf_lsvc_classfiy_{}'.format(sex)] = stack[:, 0]    
# 添加设备 id 列
df_stack['device_id']=all_id
# 保存到 csv 文件中
df_stack.to_csv('./Demo/tfidf_classfiy.csv', index=None, encoding='utf8')    

lr stacking
stack:1/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


得分0.3444655534446555
stack:2/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


得分0.3478
stack:3/5


...

得分0.3481
stack:4/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


得分0.352
stack:5/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


得分0.3714371437143714
SGDClassifier stacking
stack:1/5
得分0.33486651334866513
stack:2/5
得分0.3361
stack:3/5
得分0.3379
stack:4/5
得分0.3369
stack:5/5
得分0.35313531353135313
PassiveAggressiveClassifier stacking
stack:1/5
[0.22603554 0.40080821 0.68611114 ... 0.22700054 0.92438757 0.11568248]
得分0.37716228377162286
stack:2/5
[0.23937409 0.29277283 0.65401563 ... 0.45811155 0.69891796 0.09252268]
得分0.41
stack:3/5
[0.19112784 0.58984072 0.65485523 ... 0.65510191 0.56784305 0.19348972]
得分0.3908
stack:4/5
[0.33016848 0.32924396 0.13881066 ... 0.4053131  0.39158015 0.7748465 ]
得分0.4025
stack:5/5
[0.03220108 0.84593773 0.88390923 ... 0.73196673 0.43003357 0.34898293]
得分0.4172417241724172
RidgeClassfiy stacking
stack:1/5
[0.54860796 0.40482193 0.47736859 ... 0.42737929 0.66225863 0.37288188]
得分0.3374662533746625
stack:2/5
[0.37303388 0.38652208 0.53700431 ... 0.45571546 0.46636579 0.26696679]
得分0.3422
stack:3/5
[0.42362116 0.48107678 0.40384204 ... 0.49729276 0.47427299 0.35256109]
得分0.3469
stack:4/5
[0

In [66]:
tfidf_feat=pd.read_csv("./Demo/tfidf_classfiy.csv")

In [67]:
# 将性别预测结果和训练集、测试集合并
train_data = pd.merge(deviceid_train,tfidf_feat,on="device_id",how="left")
test_data = pd.merge(deviceid_test,tfidf_feat,on="device_id",how="left")

In [68]:
# 再次构造性别预测训练集
features = [x for x in train_data.columns if x not in ['device_id', 'sex',"age","label","app"]]
X=train_data[features]
Y = train_data['sex'] - 1

### 使用 xgb 预测最终的性别

In [69]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import auc, log_loss, roc_auc_score,f1_score,recall_score,precision_score
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold( n_splits=5, shuffle=True, random_state=1024)

params={
	'booster':'gbtree',
    
	'objective': 'binary:logistic',
#      'is_unbalance':'True',
# 	'scale_pos_weight': 1500.0/13458.0,
        'eval_metric': "logloss",
    
	'gamma':0.2,#0.2 is ok
	'max_depth':6,
# 	'lambda':20,
    # "alpha":5,
        'subsample':0.7,
        'colsample_bytree':0.4 ,
#         'min_child_weight':2.5, 
        'eta': 0.01,
    # 'learning_rate':0.01,
    "silent":1,
	'seed':1024,
	'nthread':12,
   
    }
num_round = 3500
early_stopping_rounds = 100

auc = []

test_sex = np.zeros((len(deviceid_test), ))
pred_sex=np.zeros((len(deviceid_train),))
for i, (train_index,valid_index) in enumerate(kv.split(X,Y)):
    tr_x = X.loc[train_index,:]
    tr_y = Y[train_index]
    valid_x = X.loc[valid_index,:]
    valid_y = Y[valid_index]    
    d_tr = xgb.DMatrix(tr_x, label=tr_y)
    d_valid = xgb.DMatrix(valid_x, label=valid_y)
    watchlist  = [(d_tr,'train'),(d_valid,'val')]
    model = xgb.train(params, d_tr, num_boost_round=5500, 
                      evals=watchlist,verbose_eval=200,
                              early_stopping_rounds=100)
    valid_pred = model.predict(d_valid)
    pred_sex[valid_index] =valid_pred                  
    a = log_loss(valid_y, valid_pred)    
    test_sex += model.predict(xgb.DMatrix(test_data[features]))/5
    print ("idx: ", i) 
    print (" loss: %.5f" % a)
#     print " gini: %.5f" % g
    auc.append(a)  
print ("mean")
print ("auc:       %s" % (sum(auc) / 5.0))    

C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-logloss:0.691722	val-logloss:0.691713
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.593042	val-logloss:0.611817
[400]	train-logloss:0.574775	val-logloss:0.612024
Stopping. Best iteration:
[305]	train-logloss:0.582035	val-logloss:0.611137

idx:  0
 loss: 0.61199


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-logloss:0.691727	val-logloss:0.691861
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.595386	val-logloss:0.614948
Stopping. Best iteration:
[208]	train-logloss:0.594388	val-logloss:0.614589

idx:  1
 loss: 0.61950


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-logloss:0.691712	val-logloss:0.692172
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.594102	val-logloss:0.642882
Stopping. Best iteration:
[113]	train-logloss:0.610846	val-logloss:0.637773

idx:  2
 loss: 0.64459


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-logloss:0.691712	val-logloss:0.691948
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.59368	val-logloss:0.627033
Stopping. Best iteration:
[185]	train-logloss:0.595625	val-logloss:0.626687

idx:  3
 loss: 0.63063


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-logloss:0.691656	val-logloss:0.691937
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.59061	val-logloss:0.623712
Stopping. Best iteration:
[288]	train-logloss:0.580737	val-logloss:0.622391

idx:  4
 loss: 0.62349
mean
auc:       0.626039586467934


In [70]:
# 合并训练集、测试集的性别预测结果
train_sex = pd.DataFrame(pred_sex, columns=['sex2'])
test_sex = pd.DataFrame(test_sex, columns=['sex2'])
sex=pd.concat([train_sex,test_sex])
sex['sex1'] = 1-sex['sex2']
gc.collect()

761

### 使用 APP 的 tf-idf 预测年龄

In [71]:
df_stack = pd.DataFrame()
df_stack['device_id']=all_id['device_id']
score = deviceid_train['age']

########################### lr(LogisticRegression) ################################
print('lr stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    clf = LogisticRegression(random_state=1017, C=8)
    clf.fit(train_app[tr], score[tr])
    score_va = clf.predict_proba(train_app[va])

    score_te = clf.predict_proba(test_app)
    print('得分' + str(mean_squared_error(score[va], clf.predict(train_app[va]))))
    stack_train[va] = score_va
    stack_test+= score_te
stack_test /= n_folds

stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_lr_classfiy_{}'.format(i)] = stack[:, i]


########################### SGD(随机梯度下降) ################################
print('sgd stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    sgd = SGDClassifier(random_state=1017, loss='log')
    sgd.fit(train_app[tr], score[tr])
    score_va = sgd.predict_proba(train_app[va])
    score_te = sgd.predict_proba(test_app)
    print('得分' + str(mean_squared_error(score[va], sgd.predict(train_app[va]))))
    stack_train[va] = score_va
    stack_test+= score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_sgd_classfiy_{}'.format(i)] = stack[:, i]


########################### pac(PassiveAggressiveClassifier) ################################
print('PAC stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    pac = PassiveAggressiveClassifier(random_state=1017)
    pac.fit(train_app[tr], score[tr])
    score_va = pac._predict_proba_lr(train_app[va])
    score_te = pac._predict_proba_lr(test_app)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], pac.predict(train_app[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_pac_classfiy_{}'.format(i)] = stack[:, i]



########################### ridge(RidgeClassfiy) ################################
print('RidgeClassfiy stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    ridge = RidgeClassifier(random_state=1017)
    ridge.fit(train_app[tr], score[tr])
    score_va = ridge._predict_proba_lr(train_app[va])
    score_te = ridge._predict_proba_lr(test_app)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], ridge.predict(train_app[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_ridge_classfiy_{}'.format(i)] = stack[:, i]



########################### bnb(BernoulliNB) ################################
print('BernoulliNB stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    bnb = BernoulliNB()
    bnb.fit(train_app[tr], score[tr])
    score_va = bnb.predict_proba(train_app[va])
    score_te = bnb.predict_proba(test_app)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], bnb.predict(train_app[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_bnb_classfiy_{}'.format(i)] = stack[:, i]

########################### mnb(MultinomialNB) ################################
print('MultinomialNB stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    mnb = MultinomialNB()
    mnb.fit(train_app[tr], score[tr])
    score_va = mnb.predict_proba(train_app[va])
    score_te = mnb.predict_proba(test_app)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], mnb.predict(train_app[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_mnb_classfiy_{}'.format(i)] = stack[:, i]


############################ Linersvc(LinerSVC) ################################
print('LinerSVC stacking')
stack_train = np.zeros((len(deviceid_train), 11))
stack_test = np.zeros((len(deviceid_test), 11))
score_va = 0

for i, (tr,va) in enumerate(kv.split(train_app,score)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    lsvc = LinearSVC(random_state=1017)
    lsvc.fit(train_app[tr], score[tr])
    score_va = lsvc._predict_proba_lr(train_app[va])
    score_te = lsvc._predict_proba_lr(test_app)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], lsvc.predict(train_app[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])

for i in range(stack.shape[1]):
    df_stack['pack_tfidf_lsvc_classfiy_{}'.format(i)] = stack[:, i]
    
df_stack.to_csv('./Demo/pack_tfidf_age.csv', index=None, encoding='utf8')

lr stacking
stack:1/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


得分7.486756621689155
stack:2/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


得分6.891032690192942
stack:3/5


...

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


得分7.301890567170151
stack:5/5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


得分7.4786393196598295
sgd stacking
stack:1/5
得分7.051974012993504
stack:2/5
得分6.294111766470059
stack:3/5
得分6.955
stack:4/5
得分6.659497849354806
stack:5/5
得分6.811905952976488
PAC stacking
stack:1/5
[[0.07713337 0.06078497 0.16951103 ... 0.06231956 0.05098182 0.02156769]
 [0.04089013 0.03665824 0.04973171 ... 0.09544697 0.10504735 0.11336235]
 [0.04144628 0.0696375  0.07760912 ... 0.05934811 0.0120289  0.16582504]
 ...
 [0.07056763 0.01070354 0.0260266  ... 0.02990727 0.02982439 0.10643443]
 [0.13860084 0.28810021 0.09920855 ... 0.07184668 0.03201097 0.0181112 ]
 [0.08296375 0.1447091  0.02837199 ... 0.12883953 0.03925138 0.04625413]]
得分8.996901549225388
stack:2/5
[[0.02417947 0.08456413 0.12493812 ... 0.0769574  0.06493059 0.1111957 ]
 [0.03053238 0.01633659 0.07832188 ... 0.02743429 0.11494769 0.09164805]
 [0.04752178 0.03683984 0.08151302 ... 0.10648105 0.07438747 0.08463277]
 ...
 [0.15426714 0.13663693 0.01552546 ... 0.03922786 0.04246156 0.04572334]
 [0.04546912 0.01719822 0.11075522

  9.56903709e-04 1.18486389e-02]]
得分6.410794602698651
stack:2/5
[[1.57491980e-06 1.41652236e-03 2.51789002e-02 ... 3.53685460e-02
  5.23901685e-03 3.78292472e-02]
 [3.92615955e-06 1.79113239e-03 2.11122499e-02 ... 4.71180132e-02
  2.44356017e-02 8.69246436e-02]
 [3.48729320e-05 2.58794514e-03 2.10664459e-02 ... 1.05757091e-01
  3.70895168e-02 9.02131628e-02]
 ...
 [1.14783466e-04 2.89874496e-02 4.21409881e-02 ... 6.07219929e-03
  1.70108136e-03 9.42463189e-03]
 [3.20148514e-04 1.99691425e-02 8.48070322e-02 ... 4.12726780e-02
  9.46464780e-03 3.16609227e-02]
 [2.98180013e-04 1.22697543e-02 2.30495765e-02 ... 2.11435430e-02
  8.90848031e-03 3.39600898e-02]]
得分6.145256423073078
stack:3/5
[[3.43480584e-05 7.26233731e-03 4.99491236e-02 ... 4.28083983e-02
  7.83186441e-03 2.14004797e-02]
 [4.18121523e-06 4.43896859e-03 4.93356287e-02 ... 3.23462077e-02
  6.38398827e-03 3.09554526e-02]
 [7.02596959e-04 2.08965378e-02 1.04334994e-01 ... 3.34353936e-02
  1.13390526e-02 4.31474939e-02]
 ...
 [2.

In [72]:
# 拆分训练集、测试集年龄与预测的结果
age=pd.read_csv("./Demo/pack_tfidf_age.csv")
train_data = pd.merge(deviceid_train,age,on="device_id",how="left")
test_data = pd.merge(deviceid_test,age,on="device_id",how="left")
features = [x for x in train_data.columns if x not in ['device_id',"age","sex","label","app"]]
X=train_data[features]
Y = train_data['age']
del package_time
gc.collect()

230

In [73]:
# 使用 xgb 预测最终的年龄
params={
	'booster':'gbtree',
	'objective': 'multi:softprob',
#      'is_unbalance':'True',
# 	'scale_pos_weight': 1500.0/13458.0,
        'eval_metric': "mlogloss",
    'num_class':11,
	'gamma':0.1,#0.2 is ok
	'max_depth':6,
# 	'lambda':20,
    # "alpha":5,
        'subsample':0.7,
        'colsample_bytree':0.4 ,
        # 'min_child_weight':2.5, 
        'eta': 0.01,
    # 'learning_rate':0.01,
    "silent":1,
	'seed':1024,
	'nthread':12,
    }

auc = [] 
test_age = np.zeros((len(deviceid_test),11 ))
pred_age=np.zeros((len(deviceid_train),11))
for i, (train_index,valid_index) in enumerate(kv.split(X,Y)):
    tr_x = X.loc[train_index,:]
    tr_y = Y[train_index]
    valid_x = X.loc[valid_index,:]
    valid_y = Y[valid_index]    
    d_tr = xgb.DMatrix(tr_x, label=tr_y)
    d_valid = xgb.DMatrix(valid_x, label=valid_y)
    watchlist  = [(d_tr,'train'),(d_valid,'val')]
    model = xgb.train(params, d_tr, num_boost_round=5500, 
                      evals=watchlist,verbose_eval=200,
                              early_stopping_rounds=100)
    pred = model.predict(d_valid)
    pred_age[valid_index] =pred                  
    a = log_loss(valid_y, pred)    
    test_age += model.predict(xgb.DMatrix(test_data[features]))/5
    print ("idx: ", i) 
    print (" loss: %.5f" % a)
#     print " gini: %.5f" % g
    auc.append(a)  
print ("mean")
print ("auc:       %s" % (sum(auc) / 5.0))    

C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:2.39261	val-mlogloss:2.39375
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.92146	val-mlogloss:2.07677
[400]	train-mlogloss:1.76388	val-mlogloss:2.02748
[600]	train-mlogloss:1.66875	val-mlogloss:2.02164
Stopping. Best iteration:
[553]	train-mlogloss:1.6883	val-mlogloss:2.02085

idx:  0
 loss: 2.02185


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:2.39273	val-mlogloss:2.3935
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.93122	val-mlogloss:2.06083
[400]	train-mlogloss:1.78117	val-mlogloss:2.02234
Stopping. Best iteration:
[408]	train-mlogloss:1.77686	val-mlogloss:2.02197

idx:  1
 loss: 2.02630


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:2.39258	val-mlogloss:2.39375
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.92154	val-mlogloss:2.08548
[400]	train-mlogloss:1.77097	val-mlogloss:2.05639
Stopping. Best iteration:
[364]	train-mlogloss:1.79072	val-mlogloss:2.05576

idx:  2
 loss: 2.05991


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:2.39265	val-mlogloss:2.3937
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.91947	val-mlogloss:2.07561
[400]	train-mlogloss:1.7624	val-mlogloss:2.02614
[600]	train-mlogloss:1.67019	val-mlogloss:2.02076
Stopping. Best iteration:
[561]	train-mlogloss:1.68571	val-mlogloss:2.02043

idx:  3
 loss: 2.02171


C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:2.39258	val-mlogloss:2.39377
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.91761	val-mlogloss:2.07628
[400]	train-mlogloss:1.75807	val-mlogloss:2.02423
[600]	train-mlogloss:1.66239	val-mlogloss:2.01306
[800]	train-mlogloss:1.58997	val-mlogloss:2.01078
Stopping. Best iteration:
[824]	train-mlogloss:1.58197	val-mlogloss:2.01067

idx:  4
 loss: 2.01098
mean
auc:       2.028149457113755


In [74]:
# 合并训练集、测试集的预测年龄
age=np.vstack((pred_age,test_age))
age = pd.DataFrame(age)

age.index=range(len(age))
sex.index=range(len(sex))

# 计算每个[性别-年龄]的概率
# age1 表示性别为 0 的各年龄层分布概率
# age2 表示性别为 1 的各年龄层分布概率
sex_age1=age.copy()
sex_age2=age.copy()
for i in range(11):
    sex_age1[i]=sex['sex1']*age[i]
    sex_age2[i]=sex['sex2']*age[i]

In [75]:
# 加上设备 id 列 
final_pred = pd.concat([sex_age1,sex_age2],1)
all_id.columns= ['DeviceID']
final=pd.concat([all_id,final_pred],1)
final.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', 
         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', 
         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']

# 取出测试集的预测结果，保存到 csv 中
final[50000:].to_csv('./Demo/xgb_pred.csv', index=False)

In [76]:
# 删除一些变量，回收内存
del deviceid_train
del deviceid_test
del user
gc.collect()

421

In [77]:
memory_df = get_memory()
memory_df

,name,memory,convert_memory
20,df_app,6831678012,6GB
5,atime,174103998,166MB
31,train_data,157459034,150MB
33,X,132960744,127MB
35,tr_x,105333189,100MB
16,use_time_top100_statis,100684892,96MB
32,test_data,70940379,68MB
3,deviceid_packages,65227140,62MB
17,top100_statis,63868579,61MB
25,df_stack,51272687,49MB


In [78]:
del df_app
gc.collect()

795

## 增强版：使用神经网络训练模型

In [8]:
# 读取之前处理好的特征
behave_train = pd.read_csv('./Demo/train_statistic_feat.csv')
behave_test = pd.read_csv('./Demo/test_statistic_feat.csv')

# # 设置列不限制数量
# pd.set_option('display.max_columns',None)
# # 设置行不限制数量
# pd.set_option('display.max_rows',None)
# # 由于 behave_train.columns 显示不全，因此转换为 np.array 再显示
# # print(np.array(behave_train.columns))
# # behave_train['app']
# # 由于这一列是字符串，故删除
# del behave_train['app']

behave_train.drop(['sex', 'age', 'label', 'app'], 1, inplace=True)
behave_test.drop(['sex', 'age', 'label', 'app'], 1, inplace=True)

# 读取 设备信息，包括品牌和型号
brand=pd.read_csv('./Demo/deviceid_brand.tsv',sep='\t', names=['device_id','brand','model'])
# 读取训练数据集
train=pd.read_csv('./Demo/deviceid_train.tsv',sep='\t',names=['device_id','sex','age'])
# 读取测试数据集
test=pd.read_csv('./Demo/deviceid_test.tsv',sep='\t',names=['device_id'])
# 读取 APP 使用情况
# 读取设备安装的 app 数据
packages=pd.read_csv('./Demo/deviceid_packages.tsv',sep='\t', names=['device_id','apps'])


In [9]:
# 拼接品牌和型号，和测试集、训练集合并
brand['phone_version'] = brand['brand'] + ' ' + brand['model']
train = pd.merge(brand[['device_id', 'phone_version']],
                 train, on='device_id', how='right')
test = pd.merge(brand[['device_id', 'phone_version']],
                test, on='device_id', how='right')

In [10]:
# 把用户行为数据合并到训练集、测试集
train = pd.merge(train, behave_train, on='device_id', how='left')
test = pd.merge(test, behave_test, on='device_id', how='left')

In [11]:
# 处理app，为下面分词做准备
gc.collect()
packages['app_lenghth'] = packages['apps'].apply(
    lambda x: x.split(',')).apply(lambda x: len(x))
packages['app_list'] = packages['apps'].apply(lambda x: x.split(','))
# 把app 数据合并到训练集、测试集，下面转换为词索引
train = pd.merge(train, packages, on='device_id', how='left')
test = pd.merge(test, packages, on='device_id', how='left')

In [12]:
packages.head()

,device_id,apps,app_lenghth,app_list
0,00009270c4ec26e1d76f5d86847009c9,"1896072db9ce6406febfc17f681c2086,90cb852cf345e...",3,"[1896072db9ce6406febfc17f681c2086, 90cb852cf34..."
1,000189ef5d5b951841d416a8c6c5b995,"1896072db9ce6406febfc17f681c2086,97d0422a3317b...",19,"[1896072db9ce6406febfc17f681c2086, 97d0422a331..."
2,00026d79a6f0955fc860947724e24765,c33b35d6254ad9c0c238233eb97a6c60,1,[c33b35d6254ad9c0c238233eb97a6c60]
3,0002e3afb8146bc08e40575e45f0eca6,"1896072db9ce6406febfc17f681c2086,07e967d75aab2...",3,"[1896072db9ce6406febfc17f681c2086, 07e967d75aa..."
4,0004709a296f9b925ae283efe2f043e7,"4538778ad75aa8ce61c9d13fb9cb661b,86f9f299cdbc8...",16,"[4538778ad75aa8ce61c9d13fb9cb661b, 86f9f299cdb..."


In [13]:
# 创建 Word2Vec，训练 APP 的词向量
embed_size = 128
fastmodel = Word2Vec(list(packages['app_list']), size=embed_size, window=4, min_count=3, negative=2,
                     sg=1, sample=0.002, hs=1, workers=4)

In [14]:
# 将词向量转换为 DataFrame
# 每行对应一个 APP 的词向量
embedding_fast=pd.DataFrame([fastmodel[word] for word in fastmodel.wv.vocab])
embedding_fast['app'] = list(fastmodel.wv.vocab)
embedding_fast.columns = ["fdim_%s" % str(i) for i in range(embed_size)]+['app']
embedding_fast.head()

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,fdim_0,fdim_1,fdim_2,fdim_3,fdim_4,fdim_5,fdim_6,fdim_7,fdim_8,fdim_9,...,fdim_119,fdim_120,fdim_121,fdim_122,fdim_123,fdim_124,fdim_125,fdim_126,fdim_127,app
0,-0.121516,0.372894,-0.245987,0.132014,-0.001482,-0.068758,-0.092777,0.200586,-0.134259,0.064190,...,-0.582489,-0.314622,-0.197775,-0.016513,0.417600,0.397454,0.017911,-0.195148,-0.190888,1896072db9ce6406febfc17f681c2086
1,-0.064080,0.218942,-0.037752,0.016974,-0.039094,0.009881,-0.371990,0.021120,-0.251649,-0.081215,...,-0.009384,-0.123432,0.077276,-0.002812,0.371476,0.270820,0.099713,-0.193485,-0.113203,90cb852cf345e04d508fe03f74089183
2,0.076992,0.167960,-0.197834,-0.066358,-0.264205,-0.183335,-0.049631,-0.117933,-0.180492,0.099059,...,0.168310,-0.295977,-0.334155,-0.121751,0.094657,0.059842,-0.004529,0.093666,0.058432,8c8544b6c129ad4a431be753143ed1c3
3,-0.035663,0.049326,-0.220225,0.034521,-0.261449,-0.296225,-0.389785,0.019488,-0.000664,-0.047799,...,-0.058925,0.132131,-0.012750,-0.303868,-0.242535,0.129828,0.072585,0.262386,-0.171694,97d0422a3317b1929926dc90cda4fc53
4,0.288690,0.112326,-0.097046,0.151983,-0.201054,-0.122145,-0.200348,0.004932,-0.016378,-0.054769,...,-0.120937,-0.107319,0.034874,-0.216375,0.130095,0.094428,-0.070014,-0.050806,-0.151717,5e8709466b22da6b45bfd30825bd3620


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras import optimizers

In [16]:
# 抽取出 APP 列表，填充使得长度均为 50
tokenizer = Tokenizer(lower=False, char_level=False, split=',')

tokenizer.fit_on_texts(list(packages['apps']))
# 使用 texts_to_sequences 将 APP 转换为数字
X_seq = tokenizer.texts_to_sequences(train['apps'])
X_test_seq = tokenizer.texts_to_sequences(test['apps'])

maxlen = 50
X_app = pad_sequences(X_seq, maxlen=maxlen, value=0)
X_app_test = pad_sequences(X_test_seq, maxlen=maxlen, value=0)


In [17]:
# 查看某个设备安装的 APP
X_app[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,
        11,   7,   5,  24,   4,   6,  51,   9,  14,  35, 780])

In [18]:
# 总共有 35000 个不同 APP
len(tokenizer.word_index)

35000

In [19]:
# 构建性别标签
Y_sex = train['sex']-1

In [20]:
# 根据 tokenizer 的索引，创建词向量矩阵。用于后面的 Embedding 层
# 总共有 35000 个不同 APP，由于 tokenizer 的索引从 1 开始，因此为 35001
max_feaures = 35001
embedding_matrix = np.zeros((max_feaures, embed_size))
for word in tokenizer.word_index:
    if word not in fastmodel.wv.vocab:
        continue
    embedding_matrix[tokenizer.word_index[word]]= fastmodel[word]

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [21]:
# 分别对训练集、测试集的用户行为数据做过滤，根据训练集、测试集的设备 id 过滤
behave_train = pd.merge(train[['device_id']],
                        behave_train, on='device_id', how="left")
behave_test = pd.merge(test[['device_id']],
                       behave_test, on='device_id', how="left")

In [22]:
# 取出数值矩阵，去掉第一列：设备 ID
X_behave = behave_train.iloc[:, 1:].values
X_behave_test = behave_test.iloc[:, 1:].values

In [25]:
# 定义预测性别的模型
def model_conv1D(embedding_matrix):
    K.clear_session()
    # The embedding layer containing the word vectors
    # embedding 层是根据词取出对应的词向量
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],# 有多少个词
        output_dim=embedding_matrix.shape[1], # 词向量的维度
        weights=[embedding_matrix], # 词向量矩阵
        input_length=maxlen,# 最多有 50 个词
        trainable=False
    )
    # units=128 就是输出层的维度,return_sequences=True 表示中间的状态也保留
    # input(batch_size, num_words, dim), output(batch_size, num_words, units*2)
    lstm_layer = Bidirectional(GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))    
    # 1D convolutions that can iterate over the word vectors
    
    # output (batch_size, num_words-kernel_size+1, filters)
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    
    # Define inputs
    seq = Input(shape=(maxlen,))  
    # Run inputs through embedding
    emb = emb_layer(seq)    
    
    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    # output (batch_size, 1, channels(units)) (128,128)
    gap1a = GlobalAveragePooling1D()(conv1a)
    # output (batch_size, 1, channels(units)) (128,128)
    gmp1a = GlobalMaxPool1D()(conv1a)    
    
    # 385 对应于 X_behave 的维度
    hin = Input(shape=(385, ))
    htime = Dense(64, activation='relu')(hin) # (128, 64)
    merge1 = concatenate([gap1a, gmp1a, htime])    # (128, 320) 128+128+64=320
    
    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation='sigmoid')(x)    
    
    # model = Model(inputs=[seq1, seq2, magic_input, distance_input], outputs=pred)
    model = Model(inputs=[seq, hin], outputs=pred)    
    
    model.compile(loss='binary_crossentropy',
                      optimizer=Adam())
#     model.summary()
    return model    


In [26]:
model_sex = model_conv1D(embedding_matrix)
model_sex.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 128)      4480128     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 50, 256)      197376      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 50, 128)      32896       bidirectional_1[0][0]            
__________________________________________________________________________________________________
input_2 (I

In [ ]:
if not os.path.exists('./model'):
    os.makedirs('./model')

In [28]:
# 训练模型，预测性别
kfold = StratifiedKFold(n_splits=5, random_state=20, shuffle=True)

text_sex = np.zeros((test.shape[0], ))
train_sex = np.zeros((train.shape[0], 1))
score = []

for i, (train_index, valid_index) in enumerate(kfold.split(X_app, Y_sex)):
    print("FOLD | ", i+1)
    filepath = "model/sex_weights_best_%d.h5" % i
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001, verbose=0)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=6, verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    
    model_sex = model_conv1D(embedding_matrix)
    X_app_train = X_app[train_index]
    x_app_valid = X_app[valid_index]
    x_behave_train = X_behave[train_index]
    x_behave_valid = X_behave[valid_index]
    y_train = Y_sex[train_index]
    y_valid = Y_sex[valid_index]    
    hist = model_sex.fit([X_app_train, x_behave_train], y_train, batch_size=128, epochs=50, validation_data=([x_app_valid, x_behave_valid], y_valid),
                         callbacks=callbacks, verbose=1, shuffle=True)   
    if os.path.exists(filepath):
        model_sex.load_weights(filepath)
    else:
        model_sex.save_weights(filepath)
    text_sex += np.squeeze(model_sex.predict([X_app_test, X_behave_test]))/kfold.n_splits    
    train_sex[valid_index] = model_sex.predict([x_app_valid, x_behave_valid])
    score.append(np.min(hist.history['val_loss']))    
    print('log loss:', np.mean(score))

FOLD |  1
Train on 39999 samples, validate on 10001 samples
Epoch 1/50
39999/39999 [==============================] - ETA: 35:20 - loss: na - ETA: 27:09 - loss: na - ETA: 24:20 - loss: na - ETA: 23:12 - loss: na - ETA: 22:07 - loss: na - ETA: 21:39 - loss: na - ETA: 21:19 - loss: na - ETA: 21:02 - loss: na - ETA: 20:44 - loss: na - ETA: 20:28 - loss: na - ETA: 20:12 - loss: na - ETA: 19:55 - loss: na - ETA: 19:45 - loss: na - ETA: 19:37 - loss: na - ETA: 19:30 - loss: na - ETA: 19:24 - loss: na - ETA: 19:13 - loss: na - ETA: 19:05 - loss: na - ETA: 18:56 - loss: na - ETA: 18:50 - loss: na - ETA: 18:43 - loss: na - ETA: 18:37 - loss: na - ETA: 18:30 - loss: na - ETA: 18:23 - loss: na - ETA: 18:18 - loss: na - ETA: 18:11 - loss: na - ETA: 18:06 - loss: na - ETA: 17:59 - loss: na - ETA: 17:54 - loss: na - ETA: 17:49 - loss: na - ETA: 17:45 - loss: na - ETA: 17:40 - loss: na - ETA: 17:35 - loss: na - ETA: 17:29 - loss: na - ETA: 17:24 - loss: na - ETA: 17:19 - loss: na - ETA: 17:14 - loss:

C:\Users\lenovo\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):



Epoch 00001: val_loss did not improve from inf


C:\Users\lenovo\Anaconda3\lib\site-packages\keras\callbacks.py:1024: RuntimeWarning: invalid value encountered in less
  self.monitor_op = lambda a, b: np.less(a, b - self.min_delta)
C:\Users\lenovo\Anaconda3\lib\site-packages\keras\callbacks.py:538: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


log loss: nan
FOLD |  2
Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - ETA: 32:24 - loss: na - ETA: 25:26 - loss: na - ETA: 23:14 - loss: na - ETA: 22:05 - loss: na - ETA: 21:28 - loss: na - ETA: 20:59 - loss: na - ETA: 20:36 - loss: na - ETA: 20:21 - loss: na - ETA: 20:02 - loss: na - ETA: 19:51 - loss: na - ETA: 19:44 - loss: na - ETA: 19:36 - loss: na - ETA: 19:28 - loss: na - ETA: 19:20 - loss: na - ETA: 19:11 - loss: na - ETA: 19:06 - loss: na - ETA: 18:58 - loss: na - ETA: 18:50 - loss: na - ETA: 18:43 - loss: na - ETA: 18:40 - loss: na - ETA: 18:32 - loss: na - ETA: 18:26 - loss: na - ETA: 18:19 - loss: na - ETA: 18:16 - loss: na - ETA: 18:11 - loss: na - ETA: 18:06 - loss: na - ETA: 18:01 - loss: na - ETA: 17:57 - loss: na - ETA: 17:53 - loss: na - ETA: 17:48 - loss: na - ETA: 17:44 - loss: na - ETA: 17:40 - loss: na - ETA: 17:34 - loss: na - ETA: 17:30 - loss: na - ETA: 17:25 - loss: na - ETA: 17:22 - loss: na - ETA:

40000/40000 [==============================] - ETA: 33:30 - loss: na - ETA: 26:23 - loss: na - ETA: 24:01 - loss: na - ETA: 22:36 - loss: na - ETA: 21:58 - loss: na - ETA: 21:30 - loss: na - ETA: 21:09 - loss: na - ETA: 20:52 - loss: na - ETA: 20:34 - loss: na - ETA: 20:23 - loss: na - ETA: 20:14 - loss: na - ETA: 20:05 - loss: na - ETA: 19:56 - loss: na - ETA: 19:44 - loss: na - ETA: 19:36 - loss: na - ETA: 19:27 - loss: na - ETA: 19:19 - loss: na - ETA: 19:13 - loss: na - ETA: 19:05 - loss: na - ETA: 19:01 - loss: na - ETA: 18:58 - loss: na - ETA: 18:51 - loss: na - ETA: 18:44 - loss: na - ETA: 18:39 - loss: na - ETA: 18:36 - loss: na - ETA: 18:30 - loss: na - ETA: 18:24 - loss: na - ETA: 18:19 - loss: na - ETA: 18:13 - loss: na - ETA: 18:08 - loss: na - ETA: 18:04 - loss: na - ETA: 17:59 - loss: na - ETA: 17:54 - loss: na - ETA: 17:48 - loss: na - ETA: 17:44 - loss: na - ETA: 17:40 - loss: na - ETA: 17:35 - loss: na - ETA: 17:30 - loss: na - ETA: 17:25 - loss: na - ETA: 17:21 - loss

40000/40000 [==============================] - ETA: 32:47 - loss: na - ETA: 26:16 - loss: na - ETA: 23:57 - loss: na - ETA: 22:43 - loss: na - ETA: 21:59 - loss: na - ETA: 21:31 - loss: na - ETA: 21:05 - loss: na - ETA: 20:44 - loss: na - ETA: 20:29 - loss: na - ETA: 20:14 - loss: na - ETA: 20:05 - loss: na - ETA: 19:56 - loss: na - ETA: 19:47 - loss: na - ETA: 19:37 - loss: na - ETA: 19:25 - loss: na - ETA: 19:17 - loss: na - ETA: 19:11 - loss: na - ETA: 19:04 - loss: na - ETA: 18:58 - loss: na - ETA: 18:52 - loss: na - ETA: 18:45 - loss: na - ETA: 18:39 - loss: na - ETA: 18:33 - loss: na - ETA: 18:28 - loss: na - ETA: 18:22 - loss: na - ETA: 18:20 - loss: na - ETA: 18:14 - loss: na - ETA: 18:08 - loss: na - ETA: 18:04 - loss: na - ETA: 17:58 - loss: na - ETA: 17:56 - loss: na - ETA: 17:52 - loss: na - ETA: 17:49 - loss: na - ETA: 17:45 - loss: na - ETA: 17:41 - loss: na - ETA: 17:39 - loss: na - ETA: 17:36 - loss: na - ETA: 17:32 - loss: na - ETA: 17:29 - loss: na - ETA: 17:26 - loss

40001/40001 [==============================] - ETA: 33:08 - loss: na - ETA: 26:05 - loss: na - ETA: 23:53 - loss: na - ETA: 22:42 - loss: na - ETA: 22:10 - loss: na - ETA: 21:45 - loss: na - ETA: 21:25 - loss: na - ETA: 21:07 - loss: na - ETA: 20:51 - loss: na - ETA: 20:38 - loss: na - ETA: 20:25 - loss: na - ETA: 20:14 - loss: na - ETA: 20:03 - loss: na - ETA: 19:53 - loss: na - ETA: 19:47 - loss: na - ETA: 19:36 - loss: na - ETA: 19:27 - loss: na - ETA: 19:20 - loss: na - ETA: 19:13 - loss: na - ETA: 19:07 - loss: na - ETA: 19:00 - loss: na - ETA: 18:54 - loss: na - ETA: 18:50 - loss: na - ETA: 18:44 - loss: na - ETA: 18:38 - loss: na - ETA: 18:31 - loss: na - ETA: 18:27 - loss: na - ETA: 18:22 - loss: na - ETA: 18:18 - loss: na - ETA: 18:14 - loss: na - ETA: 18:11 - loss: na - ETA: 18:08 - loss: na - ETA: 18:05 - loss: na - ETA: 18:02 - loss: na - ETA: 17:58 - loss: na - ETA: 17:54 - loss: na - ETA: 17:50 - loss: na - ETA: 17:46 - loss: na - ETA: 17:41 - loss: na - ETA: 17:38 - loss

In [29]:
# 处理预测结果，预测结果是性别为 1 的概率，根据性别为 1 的概率计算性别为 0 的概率。
text_sex = pd.DataFrame(text_sex, columns=['sex2'])
text_sex['sex1'] = 1-text_sex['sex2']

In [30]:
# 定义预测年龄的模型
def model_age_conv(embedding_matrix):

    # The embedding layer containing the word vectors
    K.clear_session()
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    )
    lstm_layer = Bidirectional(
        GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    conv2 = Conv1D(filters=64, kernel_size=2,
                   padding='same', activation='relu', )
    conv3 = Conv1D(filters=64, kernel_size=3,
                   padding='same', activation='relu',)
    conv5 = Conv1D(filters=32, kernel_size=5,
                   padding='same', activation='relu',)

    # Define inputs
    seq = Input(shape=(maxlen,))

    # Run inputs through embedding
    emb = emb_layer(seq)

    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    gap1a = GlobalAveragePooling1D()(conv1a)
    gmp1a = GlobalMaxPool1D()(conv1a)

    conv2a = conv2(lstm)
    gap2a = GlobalAveragePooling1D()(conv2a)
    gmp2a = GlobalMaxPool1D()(conv2a)

    conv3a = conv3(lstm)
    gap3a = GlobalAveragePooling1D()(conv3a)
    gmp3a = GlobalMaxPooling1D()(conv3a)

    conv5a = conv5(lstm)
    gap5a = GlobalAveragePooling1D()(conv5a)
    gmp5a = GlobalMaxPooling1D()(conv5a)

    # 385 对应于 X_behave 的维度
    hin = Input(shape=(385, ))
    htime = Dense(64, activation='relu')(hin)
    merge1 = concatenate([gap1a, gmp1a, htime])

#     merge1 = concatenate([gap1a, gap2a, gap3a, gap5a])

    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    pred = Dense(11, activation='softmax')(x)

    model = Model(inputs=[seq, hin], outputs=pred)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam())
#     model.summary()
    return model

In [ ]:
# 训练模型，预测年龄
test_age = np.zeros((X_app_test.shape[0], 11))
train_age = np.zeros((X_app.shape[0], 11))
Y_age = to_categorical(train['age'])

score = []
for i, (train_index, valid_index) in enumerate(kfold.split(X_app, train['age'])):

    print("FOLD | ", i+1)

    filepath2 = "model/age_weights_best_%d.h5" % i
    checkpoint2 = ModelCheckpoint(
        filepath2, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr2 = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001, verbose=1)
    earlystopping2 = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, mode='auto')
    callbacks2 = [checkpoint2, reduce_lr2, earlystopping2]

    model_age = model_age_conv(embedding_matrix)

    X_app_train = X_app[train_index]
    x_app_valid = X_app[valid_index]
    x_behave_train = X_behave[train_index]
    x_behave_valid = X_behave[valid_index]
    y_train = Y_age[train_index]
    y_valid = Y_age[valid_index]  


    hist = model_age.fit([X_app_train, x_behave_train], y_train, batch_size=128, epochs=50, validation_data=([x_app_valid, x_behave_valid], y_valid),
                         callbacks=callbacks2, verbose=1, shuffle=True)

    if os.path.exists(filepath2):
        model_age.load_weights(filepath2)
    else:
        model_age.save_weights(filepath2)
    train_age[valid_index] = model_age.predict([x_app_valid, x_behave_valid])
    test_age += model_age.predict([X_app_test, X_behave_test])/kfold.n_splits
    score.append(np.min(hist.history['val_loss']))
print('log loss:', np.mean(score))

In [269]:
# 根据性别和年龄的预测结果，计算各个[性别-年龄]组合的概率，作为与预测集的结果，保存到 csv 中
test_age = pd.DataFrame(test_age)
test_age1 = test_age
test_age2 = test_age
for i in range(11):
    test_age1[i] = text_sex['sex1']*test_age[i]
    test_age2[i] = text_sex['sex2']*test_age[i]
id_list=test[['device_id']]
id_list.columns = ['DeviceID']
final = pd.concat([id_list,test_age1, test_age2], 1)
final.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6',
                 '1-7', '1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4',
                 '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']
final.to_csv('./Demo/nn_pred.csv', index=False)